# Workspace API Client quickstart guide

`evo.workspaces.WorkspaceAPIClient` facilitates communication with the Workspace API. The first step is to set up the Authorizer and the APIConnector required for the SDK setup. For more information, take a look at the  `quickstart.ipynb` and `oauth.ipynb` examples.

In [1]:
from evo.aio import AioTransport
from evo.oauth import AuthorizationCodeAuthorizer, OAuthConnector, OAuthScopes

# OAuth client app credentials
# See: https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens
REDIRECT_URL = "http://localhost:3000/signin-oidc"
CLIENT_NAME = "Evo Python SDK"
CLIENT_ID = "evopythonsdk"

transport = AioTransport(
    user_agent=CLIENT_NAME,
)

connector = OAuthConnector(
    transport=transport,
    client_id=CLIENT_ID,
    base_uri="https://qa-imsoidc.bentley.com",
)

authorizer = AuthorizationCodeAuthorizer(
    oauth_connector=connector,
    redirect_url=REDIRECT_URL,
    scopes=OAuthScopes.evo_discovery | OAuthScopes.evo_workspace,
)

# Login to the Evo platform.
await authorizer.login(timeout_seconds=15)

## WorkspaceAPIClient

Now that the `authorizer` is set up, we can set up the `WorkspaceServiceClient`. This client wraps endpoint functionality to provide a cohesive interface to the underlying API client implementation.

For initialization the client requires:
* The organization ID since all workspace service operations are currently bound by this ID.
* The hub URL to connect to the workspace service.

In [2]:
import uuid

from evo.common import APIConnector
from evo.workspaces import WorkspaceAPIClient

# See "quickstart.ipynb" for assistance in obtaining your organization ID and hub URL
ORG_ID = uuid.UUID("e4301b4b-bd8f-42e8-82f7-e26c0f1c3f5b")
HUB_URL = "https://350mt.api.integration.seequent.com"

workspace_client = WorkspaceAPIClient(
    connector=APIConnector(HUB_URL, transport, authorizer),
    org_id=ORG_ID,
)

service_health = await workspace_client.get_service_health()

print(f"Workspace API is {service_health.status.name.lower()}")

Workspace API is healthy


## Workspace operations
In this section we will go over some of the available workspace operations.

### Listing workspaces
To list the workspaces you can either use `WorkspaceAPIClient.list_workspaces()` method:

In [ ]:
from evo.common import Page
from evo.workspaces import Workspace

paginated_workspaces: Page[Workspace] = await workspace_client.list_workspaces(limit=5, offset=0)
for item in paginated_workspaces.items():
    print(item)


The `list_workspaces` will return a `evo.common.Page` object that encapsulates the paginated response by the Workspace Service.
Alternatively, you can list all workspaces available to the given user by calling the `WorkspaceServiceClient.list_all_workspaces()` method:

In [ ]:
all_workspaces: list[Workspace] = await workspace_client.list_all_workspaces()

Both `list_workspaces` and `list_all_workspaces` support all the query parameters supported by the workspace service's [List Workspaces](https://developer.seequent.com/docs/api/workspaces/list-workspaces) endpoint:

In [ ]:
workspaces = await workspace_client.list_workspaces(
    limit=5,
    offset=0,
    deleted=False,
    order_by={"name": "asc"},
)

### Creating a workspace

In [ ]:
new_workspace: Workspace = await workspace_client.create_workspace(
    name="My New Workspace",
    description="This is a new workspace",
    bounding_box_coordinates=[
        (85.8287, -90.0),
        (85.8287, 69.84278),
        (-180.0, 69.84278),
        (-180.0, -90.0),
        (85.8287, -90.0),
    ],
    # A square bounding box. It is important to mention that the first and last coordinates must be the same.
    # Also, the first coordinate element must be the longitude and the second element must be the latitude.
)
print(new_workspace)

### Updating a workspace

In [ ]:
updated_workspace: Workspace = await workspace_client.update_workspace(
    workspace_id=new_workspace.id,
    name="My Updated Workspace",
    description="This is an updated workspace",
)

### Deleting a workspace

In [ ]:
await workspace_client.delete_workspace(workspace_id=new_workspace.id)

## Users operation

In this section we will go over some of the available user related operations.

### Get current user's workspace role

In [ ]:
from evo.workspaces import User, UserRole

user_role: UserRole = await workspace_client.get_current_user_role(workspace_id=new_workspace.id)

### Get list of users with access to a workspace.

In [ ]:
users: list[User] = await workspace_client.list_user_roles(workspace_id=new_workspace.id)

### Assign a user to a workspace

In [ ]:
from evo.workspaces import WorkspaceRole

new_users_role: UserRole = await workspace_client.assign_user_role(
    workspace_id=new_workspace.id, user_id=uuid.UUID("<some-user-id>"), role=WorkspaceRole.viewer
)

### Remove a user from a workspace

In [ ]:
await workspace_client.delete_user_role(
    workspace_id=new_workspace.id,
    user_id=uuid.UUID("<some-user-id>"),
)

### List instance users

In [3]:
response = await workspace_client.list_instance_users()
for item in response.items():
    print(item)

InstanceUserWithEmail(user_id=UUID('4aedf748-c64b-45ed-ae4b-fd5050e5b9f3'), roles=[InstanceUserRole(role_id=UUID('5affbe84-7b04-4e0e-b5d9-a561fbe9c6cc'), description='', name='Evo Owner')], email='Sachin.Seizer@bentley.com', full_name='Sachin Seizer')
InstanceUserWithEmail(user_id=UUID('29b5b07c-77c7-4281-9002-fcec51243363'), roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')], email='Bardia.Abhari@bentley.com', full_name='Bardia Abhari')
InstanceUserWithEmail(user_id=UUID('f2ecc805-7fba-4d91-9d58-d622f16a560c'), roles=[InstanceUserRole(role_id=UUID('2e5ce7d6-9e93-4b72-ac23-674d6d84e70f'), description='', name='Evo Admin')], email='Samiul.Azam@bentley.com', full_name='Samiul Azam')
InstanceUserWithEmail(user_id=UUID('72cab684-1913-4925-b2a2-a52e93c2f0d3'), roles=[InstanceUserRole(role_id=UUID('2e5ce7d6-9e93-4b72-ac23-674d6d84e70f'), description='', name='Evo Admin')], email='Sultan.Alzoghaibi@bentley.com', full_name='Sultan Al

### List all instance users

In [4]:
response = await workspace_client.list_all_instance_users()
for item in response:
    print(item)

InstanceUserWithEmail(user_id=UUID('29b5b07c-77c7-4281-9002-fcec51243363'), roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')], email='Bardia.Abhari@bentley.com', full_name='Bardia Abhari')
InstanceUserWithEmail(user_id=UUID('f0b3a897-79c7-450b-997c-73f365662dfd'), roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')], email='ChunLok.Chan@bentley.com', full_name='Chun Lok Chan')
InstanceUserWithEmail(user_id=UUID('1e225589-7c00-4002-bce4-441fec8b3323'), roles=[InstanceUserRole(role_id=UUID('2e5ce7d6-9e93-4b72-ac23-674d6d84e70f'), description='', name='Evo Admin')], email='Gibson.Dziwinski@bentley.com', full_name='Gibson Dziwinski')
InstanceUserWithEmail(user_id=UUID('81ce9ca1-751d-4c22-9ea3-8addbaf2fbc0'), roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')], email='Ravleen.Anand@bentley.com', full_name='Ravlee

### Add user to instance

In [ ]:
email = 'ChunLok.Chan@bentley.com'
role_id = uuid.UUID('2894b27e-266d-46c2-892b-3ae604fffb64')
users = {email : [role_id], 'asdf3@gmail.com' : [role_id]}
response = await workspace_client.add_users_to_instance(users)
for item in response:
    print(item)

InstanceUserWithEmail(user_id=UUID('f0b3a897-79c7-450b-997c-73f365662dfd'), roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')], email='ChunLok.Chan@bentley.com', full_name='Chun Lok Chan')


### List instance user invitations

In [12]:
response = await workspace_client.list_instance_user_invitations()
for item in response:
    print(item)

InstanceUserInvitation(email='asdf@gmail.com', invitation_id=UUID('e83ea5a0-1f94-4765-8e3e-4f87069511f3'), invited_at=datetime.datetime(2026, 1, 8, 21, 54, 27, 613000, tzinfo=TzInfo(UTC)), invited_by='sachin.seizer@bentley.com', status='Pending', roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')])
InstanceUserInvitation(email='asdf@gmail.com', invitation_id=UUID('2737cf26-2169-4c0e-be73-bd55c7895ad4'), invited_at=datetime.datetime(2026, 1, 8, 22, 12, 0, 143000, tzinfo=TzInfo(UTC)), invited_by='sachin.seizer@bentley.com', status='Pending', roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')])


### List all instance user invitations

In [11]:
response = await workspace_client.list_all_instance_user_invitations()
for item in response:
    print(item)

InstanceUserInvitation(email='asdf@gmail.com', invitation_id=UUID('e83ea5a0-1f94-4765-8e3e-4f87069511f3'), invited_at=datetime.datetime(2026, 1, 8, 21, 54, 27, 613000, tzinfo=TzInfo(UTC)), invited_by='sachin.seizer@bentley.com', status='Pending', roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')])
InstanceUserInvitation(email='asdf@gmail.com', invitation_id=UUID('2737cf26-2169-4c0e-be73-bd55c7895ad4'), invited_at=datetime.datetime(2026, 1, 8, 22, 12, 0, 143000, tzinfo=TzInfo(UTC)), invited_by='sachin.seizer@bentley.com', status='Pending', roles=[InstanceUserRole(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='', name='Evo User')])


### Delete instance user invitation

In [10]:
invitation_id = uuid.UUID("9e29eb2c-4155-4ca7-b966-ad1294052f54")

await workspace_client.delete_instance_user_invitation(invitation_id=invitation_id)

### List instance user roles

In [3]:
response = await workspace_client.list_instance_user_roles()
for item in response:
    print(item)

InstanceUserRoleWithPermissions(role_id=UUID('2894b27e-266d-46c2-892b-3ae604fffb64'), description='Provide access to the Evo Instance for all users in this role', name='Evo User', permissions=['EvoViewerPermission'])
InstanceUserRoleWithPermissions(role_id=UUID('2e5ce7d6-9e93-4b72-ac23-674d6d84e70f'), description='Provide admin access to the Evo Instance for all users in this role', name='Evo Admin', permissions=['INVITEPRTCPNT', 'ROLEPRMSNEDIT', 'REMOVEPRTCPNT', 'admin_manage_groups', 'EvoAdminPermission', 'EvoViewerPermission'])


### Remove instance user

In [6]:
user_id = uuid.UUID("f0b3a897-79c7-450b-997c-73f365662dfd")

await workspace_client.remove_instance_user(user_id=user_id)

### Update instance user roles

In [4]:
user_id = uuid.UUID("f0b3a897-79c7-450b-997c-73f365662dfd")
roles = [uuid.UUID("2e5ce7d6-9e93-4b72-ac23-674d6d84e70f")]

response = await workspace_client.update_instance_user_roles(user_id=user_id, roles=roles)

print(response)

InstanceUser(user_id=UUID('f0b3a897-79c7-450b-997c-73f365662dfd'), roles=[InstanceUserRole(role_id=UUID('2e5ce7d6-9e93-4b72-ac23-674d6d84e70f'), description='', name='Evo Admin')])
